In [31]:
#compute the eigenvectors of Schur's matrix according to Morton's '78 paper, "On the Eigenvectors of Schur's Matrix"
#https://www.sciencedirect.com/science/article/pii/0022314X80900839

In [40]:
#compute the eigenvectors of Schur's matrix (symbolically) over the complex numbers (or a cyclotomic field containing q^th roots of unity)
#note: A is not unitary 
q=3
alpha = exp(2*pi*I/q)
#K.<alpha> = CyclotomicField(3)
A = matrix([[(alpha**(i*j)).expand() for j in range(1,q+1)] for i in range(1,q+1)])
A.eigenvectors_right()

[(I*sqrt(3), [(1, -1, 0)], 1),
 (-sqrt(3), [(1, 1, -sqrt(3) + 1)], 1),
 (sqrt(3), [(1, 1, sqrt(3) + 1)], 1)]

In [44]:
#compute the eigenvectors of the DFT matrix (symbolically) over the complex numbers (or a cyclotomic field containing q^th roots of unity)
#NOTE: The DFT matrix is subtly different than the Schur matrix
#DFT = exp(2*pi*i*m*n/q) for 0 <= m,n <= q-1
#whereas for Schur's matrix A it is 1 <= m,n <= q
#this amounts to applying the cyclic permutation \sigma = (1 2 ... n) to Schur's matrix to both rows and columns
#if the row permutation is P_\sigma, the column permutation is P_\sigma^{T} = P_\sigma^{-1}
#then DFT = P_\sigma*A*P_\sigma^{-1}, so the two matrices are similar and have the same eigenvalues
#the eigenvectors of DFT are Pv if v is an eigenvector of A, i.e. cyclicly permuting the entries of v
DFT = matrix([[(alpha**(i*j)).expand() for j in range(q)] for i in range(q)])
DFT.eigenvectors_right()

[(I*sqrt(3), [(0, 1, -1)], 1),
 (-sqrt(3), [(1, -1/2*sqrt(3) - 1/2, -1/2*sqrt(3) - 1/2)], 1),
 (sqrt(3), [(1, 1/2*sqrt(3) - 1/2, 1/2*sqrt(3) - 1/2)], 1)]

In [72]:
#check that A is similar to the DFT matrix by cyclicly permuting columns and rows
sigma = Permutation([(i % q)+1 for i in range(1,q+1)])
P = sigma.to_matrix()
P.inverse()*DFT*P == A

True

In [92]:
#define the vectors X_d given a divisor d of q/f(chi), where f(chi) is the conductor of the character \chi
def X(d,chi):
    q = chi.modulus()
    assert d.divides(q/chi.conductor())
    return vector([chi.primitive_character()(n/d) if d.divides(n) else 0 for n in range(1,q+1)])

In [91]:
#compute eigenvectors given a character chi and a divisor d of q/f(chi)
def eigenvector(d,chi):
    try:
        assert d.divides(q/chi.conductor())
    except AssertionError:
        print("Error: d must divide q/f(chi)")
        return

    #define the eigenvectors E(chi,d,lambda) as in (13) in the paper
    conjugate_divisor = Integer(chi.modulus()/(chi.conductor()*d))
    eig_plus = sqrt(chi(-1)*chi.modulus())
    eig_minus = -sqrt(chi(-1)*chi.modulus())
    #need to restrict character to modulus conductor, i.e. the primitive character
    E = lambda chi, d, eig: sqrt(d)*X(d,chi) + (eig/(chi.bar().primitive_character().gauss_sum()*sqrt(d)))*conjugate(X(conjugate_divisor,chi))

    #check if chi is real and if the ratio of modulus to conductor is the square of a divisor
    reality_condition = chi.bar() == chi
    square_condition = d^2 == chi.modulus()/chi.conductor()

    #if chi is real and square condition is met, just return X_d(chi)
    if reality_condition and square_condition:
        return X(d,chi)

    #if one or the other condition is met, return a pair of eigenvalues corresponding to E(chi,d,\pm lambda)
    if reality_condition != square_condition:
        return E(chi,d,eig_plus), E(chi,d,eig_minus)

    #if neither condition are met, return four eigenvalues E(chi,d,\pm lambda) and E(chi_bar,d,\pm lambda)
    if (not reality_condition) and (not square_condition):
        return E(chi,d,eig_plus), E(chi,d,eig_minus), E(chi.bar(),d,eig_plus), E(chi.bar(),d,eig_minus)

In [98]:
from sage.misc.flatten import flatten
#compute all the eigenvectors the Schur matrix
def all_eigenvectors(q):
    all_eigs = []
    G = DirichletGroup(q)
    for chi in G:
        f = chi.conductor()
        for d in divisors(q/f):
            print("chi: ",chi,"| d:",d)
            all_eigs.append(eigenvector(Integer(d),chi))
    return all_eigs

In [100]:
#for q=3, it seems that the second condition is being called twice when it should only be called once
#it's giving two independent eigenvectors each time, but they are dependent
#there are also two total extra vectors, so removing these two would fix the problem

In [99]:
all_eigenvectors(3)

chi:  Dirichlet character modulo 3 of conductor 1 mapping 2 |--> 1 | d: 1
chi:  Dirichlet character modulo 3 of conductor 1 mapping 2 |--> 1 | d: 3
chi:  Dirichlet character modulo 3 of conductor 3 mapping 2 |--> -1 | d: 1


[((1, 1, sqrt(3) + 1), (1, 1, -sqrt(3) + 1)),
 ((1, 1, sqrt(3) + 1), (-1, -1, sqrt(3) - 1)),
 (1, -1, 0)]